In [ ]:
%matplotlib nbagg
import math
import numpy as np
import scipy as sp
from scipy import linalg
from scipy import integrate
import matplotlib.pyplot as plt
from matplotlib import cm

import pickle

from actuated_Koopman_EM_4 import ActuatedKoopmanEM, save_EM_model, load_EM_model

# Duffing equation

We consider the unforced Duffing equation,
\begin{equation}
    \ddot{y} = - \delta \dot{y} - y(\beta + \alpha y^2),
\end{equation}
with parameters $\delta=0.5$, $\beta=-1$, and $\alpha=1$, which were used in: 

M. O. Williams, I. G. Kevrekidis, and C. W. Rowley, ``A data-driven approximation of the Koopman operator: Extending dynamic mode decomposition'', J. Nonlinear Science, 2015

and in

S. E. Otto and C. W. Rowley, ``Linearly recurrent autoencoder networks for learning dynamics'', SIAM J. Applied Dynamical Systems, 2019

With these parameters, the Duffing equation has stable fixed points at $x=\pm 1$ with eigenvalues $\lambda_{1,2} = \frac{1}{4} \left(-1 \pm \sqrt{31} i\right)$ and a saddle point at the origin with eigenvalues $\lambda_{3,4} = \frac{1}{4} \left(-1 \pm \sqrt{17}\right)$. 

In [ ]:
# Duffing equation state dimension
n_states = 2

# number of output observables
n_outputs = 2

# paramters
delta = 0.5
beta = -1.0
alpha = 1.0

# Duffing equation as 1st order system
# x[0] = y, x[1] = dy/dt
f_sys = lambda t, x: (x[1], -delta*x[1] - x[0]*(beta + alpha*np.square(x[0])))

In [ ]:
lam_sys = np.array([-1+np.sqrt(31)*1j, \
                    -1-np.sqrt(31)*1j, \
                    -1+np.sqrt(17), \
                    -1-np.sqrt(17)]) / 4.0

## Generate data

In [ ]:
# length of trajectories and sampling interval
n_t = 801
t_data = np.linspace(0, 16, n_t)
dt = t_data[1] - t_data[0]

# initial condition domain
ICbox = [-2.0, 2.0, -2.0, 2.0]

# number of trajectories to sample
n_traj = 50

X_data = np.zeros((n_traj, n_t, n_states))
Y_data = np.zeros((n_traj, n_t, n_outputs))
U_data = np.ones((n_traj, n_t-1, 1))
for k in range(n_traj):
    # initial condition
    x0 = np.zeros(n_states)
    x0[0] = np.random.uniform(ICbox[0], ICbox[1])
    x0[1] = np.random.uniform(ICbox[2], ICbox[3])
    
    sol = sp.integrate.solve_ivp(fun = f_sys, \
                                 t_span = (t_data[0], t_data[-1]), \
                                 y0 = x0, \
                                 t_eval = t_data)
    
    X_data[k, :, :] = sol.y.T
    # Y_data[k, :, 0] = sol.y[0, :]
    Y_data[k, :, :] = sol.y[:n_outputs, :].T

# noise to be added to observation data
Noise_data = 0.01*np.random.randn(n_traj, n_t, n_outputs)

## Save data

In [ ]:
fname = 'models/Duffing/eval_data_fullobs_fine_tmp.p'

data = {'X_data': X_data, \
        'Y_data': Y_data, \
        'Noise_data': Noise_data, \
        'U_data': U_data, \
        't_data': t_data, \
        'n_traj': n_traj, \
        'n_t': n_t, \
        'dt': dt, \
        'ICbox': ICbox}

with open( fname, "wb" ) as f:
    pickle.dump(data, f)

## Load data

In [ ]:
fname = 'models/Duffing/train_data_fullobs_fine.p'

with open(fname,'rb') as f:
    data = pickle.load(f)

X_train = data['X_data']
Y_train = data['Y_data']
Noise_train = data['Noise_data']
U_train = data['U_data']
t_train = data['t_data']
n_traj_train = data['n_traj']
n_t_train = data['n_t']
dt_train = data['dt']

In [ ]:
plt.figure()
for k in range(n_traj_train):
    plt.plot(X_train[k,:,0], X_train[k,:,1], 'b.-')
    
plt.show()

# EDMD with Legendre polynomials

In [ ]:
def SpecialPolyDict(X, degree, prnt=False):
    # Construct a dictionary of special polynomials.
    # X is an n_dim by m_samples matrix
    # degree is the maximum power of each feature in X
    # returns Phi, an (degree+1)^n_dim by m_samples matrix
    #     whose rows contain all special polynomial products
    #     of the features in X up to the given degree
    
    n_dim = X.shape[0]
    m_samples = X.shape[1]
    N_dict = (degree+1)**n_dim
    
    Phi = np.ones((N_dict, m_samples))
    
    # powers is a vector containing the powers on each variable
    # the entries of powers are found by counting in base (degree + 1)
    powers = np.zeros(n_dim, dtype=int)
    if prnt:
        print(powers)
    
    for k in range(1, N_dict):
        # increment the elements of powers (by counting)
        powers[0] = powers[0] + 1
        # carry the ones
        for i in range(n_dim-1):
            if powers[i] == degree + 1:
                # carry the one
                powers[i] = 0
                powers[i+1] = powers[i+1] + 1
        
        if prnt:
            print(powers)
        
        # construct kth special monomial with powers on each variable
        # in X given by powers
        for i in range(n_dim):
            # Phi[k,:] = Phi[k,:] * sp.special.eval_hermite(powers[i], X[i,:])
            Phi[k,:] = Phi[k,:] * sp.special.eval_legendre(powers[i], X[i,:])
    
    return Phi

In [ ]:
# data matrices
X = np.vstack(X_train).T
X_Phi = np.vstack(X_train).T
X_Phi_IC = np.vstack(X_train[:,0,:]).T

# time-shifted data matrices
X_Phi0 = np.vstack(X_train[:,:-1,:]).T
X_Phi1 = np.vstack(X_train[:,1:,:]).T

# construct feature matrices
deg = 3
Phi = SpecialPolyDict(X_Phi/2.0, degree=deg, prnt=False)
Phi_IC = SpecialPolyDict(X_Phi_IC/2.0, degree=deg)
Phi0 = SpecialPolyDict(X_Phi0/2.0, degree=deg)
Phi1 = SpecialPolyDict(X_Phi1/2.0, degree=deg)

print(Phi.shape[0])

## Non-truncated EDMD

# find matrix approximation of Koopman generator
V_EDMD, residuals, _, _ = np.linalg.lstsq(Phi0.T, (Phi1.T - Phi0.T)/dt_train, rcond=None)
V_EDMD[:,0] = 0.0

mse = sum(residuals)/Phi0.shape[1]
print(mse)

# output reconstruction map
C_EDMD = np.zeros((2,Phi.shape[0]))
C_EDMD[0,1] = 2.0
C_EDMD[1,deg+1] = 2.0

# ## SVD-truncated EDMD
# r = 15
# U_Phi, s_Phi, VT_Phi = sp.linalg.svd(Phi[1:,:], full_matrices=False, compute_uv=True)
# Phi_r = np.vstack([Phi[0,:], np.reshape(s_Phi[:r-1], (-1,1))*VT_Phi[:r-1,:]])
# Phi_IC_r = np.vstack([Phi_IC[0,:], np.dot(U_Phi[:,:r-1].T, Phi_IC[1:,:])])
# Phi0_r = np.vstack([Phi0[0,:], np.dot(U_Phi[:,:r-1].T, Phi0[1:,:])])
# Phi1_r = np.vstack([Phi1[0,:], np.dot(U_Phi[:,:r-1].T, Phi1[1:,:])])

# # find matrix approximation of Koopman generator
# V_EDMD_r, residuals, _, _ = np.linalg.lstsq(Phi0_r.T, (Phi1_r.T - Phi0_r.T)/dt_train, rcond=None)
# V_EDMD_r[:,0] = 0.0

# mse = sum(residuals)/Phi0_r.shape[1]
# print(mse)

# # output reconstruction map
# C_EDMD_r = np.transpose(np.linalg.lstsq(Phi_r.T, np.vstack(Y_train), rcond=None)[0])

## Initial spectrum

In [ ]:
def eig_transform(A):
    lam, Vl, Vr = sp.linalg.eig(A, left=True, right=True)
    w = np.diag(np.dot(np.conj(Vr).T, Vl))
    Vl = Vl/w
    return lam, Vl, Vr

In [ ]:
lam_EDMD, Vl_EDMD, Vr_EDMD = eig_transform(V_EDMD.T)
# lam_EDMD, Vl_EDMD, Vr_EDMD = eig_transform(V_EDMD_r.T)

plt.figure()
plt.plot(np.real(lam_EDMD), np.imag(lam_EDMD), 'bo')
plt.plot(np.real(lam_sys), np.imag(lam_sys), 'kx')
plt.axvline(x=0, color='k')
plt.axis('equal')
plt.show()

### Eigenfunction values

In [ ]:
e_fun_vals_EDMD = np.dot(np.conj(Vl_EDMD).T, Phi)
# e_fun_vals_EDMD = np.dot(np.conj(Vl_EDMD).T, Phi_r)

efun_idcs = np.argsort(np.real(lam_EDMD))[::-1]

for i in range(len(efun_idcs)):
    print('lam_{:d} = {:f} + {:f}i'.format(i, \
                                           lam_EDMD[efun_idcs[i]].real, \
                                           lam_EDMD[efun_idcs[i]].imag))

In [ ]:
ix = 1
efun_idx = efun_idcs[ix]

# color scaling
vmean = np.mean(np.reshape(np.real(e_fun_vals_EDMD[efun_idx,:]), (-1)))
vscl = np.max(np.absolute(np.reshape(np.real(e_fun_vals_EDMD[efun_idx,:]), (-1)) - vmean))
# vscl = np.max(np.reshape(np.absolute(np.real(e_fun_vals_EDMD[efun_idx,:])), (-1)))

plt.figure()

plt.scatter(X_Phi[0,:], \
            X_Phi[1,:], \
            c = np.real(e_fun_vals_EDMD[efun_idx,:]), \
            cmap = cm.seismic, \
            vmin = vmean-vscl, \
            vmax = vmean+vscl, \
            s=10)

plt.title('phi_{:d}, lam_{:d} = {:.5f} + {:.5f}i'.format(ix, ix, lam_EDMD[efun_idx].real, lam_EDMD[efun_idx].imag))

# plt.savefig('figures/Duffing/EDMD_dim{:d}_efun{:d}_real.png'.format(V_EDMD.shape[0], ix), dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('figures/Duffing/EDMD_dim{:d}_efun{:d}_real.eps'.format(V_EDMD.shape[0], ix), dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()


# color scaling
vscl = np.max(np.reshape(np.absolute(e_fun_vals_EDMD[efun_idx,:]), (-1)))

plt.figure()

plt.scatter(X_Phi[0,:], \
            X_Phi[1,:], \
            c = np.absolute(e_fun_vals_EDMD[efun_idx,:]), \
            cmap = cm.inferno, \
            vmin = 0, \
            vmax = vscl, \
            s=10)

plt.title('|phi_{:d}|, lam_{:d} = {:.5f} + {:.5f}i'.format(ix, ix, lam_EDMD[efun_idx].real, lam_EDMD[efun_idx].imag))

# plt.savefig('figures/Duffing/EDMD_dim{:d}_efun{:d}_mag.png'.format(V_EDMD.shape[0], ix), dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('figures/Duffing/EDMD_dim{:d}_efun{:d}_mag.eps'.format(V_EDMD.shape[0], ix), dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()

# plot angle of complex eigenfunction
plt.figure()

plt.scatter(X_Phi[0,:], \
            X_Phi[1,:], \
            c = np.angle(e_fun_vals_EDMD[efun_idx,:]), \
            cmap = cm.twilight, \
            vmin = -np.pi, \
            vmax = np.pi, \
            s=10)

plt.title('arg(phi_{:d}), lam_{:d} = {:.5f} + {:.5f}i'.format(ix, ix, lam_EDMD[efun_idx].real, lam_EDMD[efun_idx].imag))

# plt.savefig('figures/Duffing/EDMD_dim{:d}_efun{:d}_arg.png'.format(V_EDMD.shape[0], ix), dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('figures/Duffing/EDMD_dim{:d}_efun{:d}_arg.eps'.format(V_EDMD.shape[0], ix), dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()

# Train Koopman EM model

## Initialize model with EDMD

In [ ]:
## initial generator matrix
V_init = V_EDMD.T

lam_init, _, _ = eig_transform(V_init)

fig, ax = plt.subplots()
ax.plot(np.real(lam_EDMD), np.imag(lam_EDMD), 'bo')
ax.plot(np.real(lam_init), np.imag(lam_init), 'gd')
ax.plot(np.real(lam_sys), np.imag(lam_sys), 'kx')
ax.axvline(x=0, color='k')
circle = plt.Circle((-1.0/dt_train, 0), radius=1.0/dt_train, edgecolor='k', facecolor='None')
ax.add_patch(circle)
plt.axis('equal')
plt.draw()

## estimate process noise covariance
E = Phi0 + dt_train*np.dot(V_init, Phi0) - Phi1
Sig_v_init = np.dot(E[1:,:], E[1:,:].T)/E.shape[1]

## estimate distribution of lifted initial states

# initialize state mean
Z = Phi_IC[1:,:].T
mu_0_EDMD = np.mean(Z, axis=0)

# initialize state covariance
Zc = Z - mu_0_EDMD
Sig_0_EDMD = np.dot(Zc.T, Zc)/Zc.shape[0]

In [ ]:
y_dim = 2
u_dim = 1
z_dim = V_init.shape[0]

# initialize generator
V_matrices = np.zeros((u_dim,z_dim,z_dim))
V_matrices[0,:,:] = V_init


# initialize process noise covariance
Sig_v = Sig_v_init

# observation matrix (fixed)
C = C_EDMD[:,1:]

# observation offset
h = C_EDMD[:,0]

# initialize observation noise covariance
Sig_w = 0.01*np.eye(y_dim)

# initialize initial state mean
mu_0 = mu_0_EDMD

# initialize initial state covariance
Sig_0 = Sig_0_EDMD

EM_model = ActuatedKoopmanEM(Y_train+Noise_train, dt_train*U_train, \
                             V_matrices, C, h, Sig_v, Sig_w, mu_0, Sig_0, \
                             compute_log_likelihood=True)

# save the model
fname = 'models/Duffing/model_dim{:d}'.format(z_dim)
save_EM_model(EM_model, fname, Y_train+0.0*Noise_train, dt_train*U_train)

fname = 'models/Duffing/EDMD_model_dim{:d}'.format(z_dim)
save_EM_model(EM_model, fname, Y_train+0.0*Noise_train, dt_train*U_train)

## Train model

In [ ]:
delta_L_tol = 1.0e-2

# number of steps between log likelihood calculations
n_skip_L_calc = 5

# model dimension
# z_dim = 16

# Load the model
fname = 'models/Duffing/model_dim{:d}'.format(z_dim)
EM_model = ActuatedKoopmanEM()
EM_model, _, _ = load_EM_model(EM_model, fname)

# log likelihood of the model
L = EM_model.L
delta_L = np.inf

## if the log likelihood has not converged continue running, 
# run the optimization
while delta_L > delta_L_tol:
    
    print('dimension {:d} iteration {:d} \n'.format(z_dim, EM_model.EM_iter+1))
    if EM_model.EM_iter%n_skip_L_calc == 0 and EM_model.EM_iter > 0:
        # run EM step and compute log likelihood
        EM_model.run_EM_step(explicit_time_step = True, optimize_IC=True, 
                             optimize_observation_map=False, optimize_process_noise=True, \
                             compute_log_likelihood=True)
        
        # average change in log likelihood
        delta_L = (EM_model.L - L)/n_skip_L_calc
        print('delta_L = {:3e}\n'.format(delta_L))
        
        # update the log likelihood
        L = EM_model.L
        
        ## Save the model
        if delta_L > 0:
            save_EM_model(EM_model, fname, Y_train+0.0*Noise_train, dt_train*U_train)
        
        # plot eigenvalues
        lam_EM, Vl_EM, Vr_EM = eig_transform(EM_model.V_matrices[0,:,:].T)
        
        fig, ax = plt.subplots()
        ax.plot(np.real(lam_EDMD), np.imag(lam_EDMD), 'bo')
        ax.plot(np.real(lam_EM), np.imag(lam_EM), 'gd')
        ax.plot(np.real(lam_sys), np.imag(lam_sys), 'kx')
        ax.axvline(x=0, color='k')
        circle = plt.Circle((-1.0/dt_train, 0), radius=1.0/dt_train, edgecolor='k', facecolor='None')
        ax.add_patch(circle)
        plt.axis('equal')
        plt.draw()
        
    else:
        ## run an EM step without computing log likelihood
        EM_model.run_EM_step(explicit_time_step = True, optimize_IC=True, \
                             optimize_observation_map=False, optimize_process_noise=True, \
                             compute_log_likelihood=False)

## Model spectrum

In [ ]:
z_dim = 16

fname = 'models/Duffing/model_dim{:d}'.format(z_dim)
EM_model = ActuatedKoopmanEM()
EM_model, _, _ = load_EM_model(EM_model, fname)

lam_EM, Vl_EM, Vr_EM = eig_transform(EM_model.V_matrices[0,:,:])

# lam_EM, Vl_EM, Vr_EM = eig_transform(V_init)

fig, ax = plt.subplots()
ax.plot(np.real(lam_EDMD), np.imag(lam_EDMD), 'gd', label='EDMD')
ax.plot(np.real(lam_EM), np.imag(lam_EM), 'bo', label='EM model')
ax.plot(np.real(lam_sys), np.imag(lam_sys), 'kx', label='Duffing fixed points')
ax.axvline(x=0, color='k')
circle = plt.Circle((-1.0/dt_train, 0), radius=1.0/dt_train, \
                    edgecolor='k', \
                    linestyle='--', \
                    facecolor='None', \
                    label='Explicit Euler stability')
ax.add_patch(circle)
# ax.set(xlim=(-8, 2), ylim=(-6, 6))
plt.legend()
plt.axis('equal')

# plt.savefig('figures/Duffing/eigenvalues_dim{:d}.png'.format(z_dim), dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('figures/Duffing/eigenvalues_dim{:d}.eps'.format(z_dim), dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()

In [ ]:
# model state estimates on training data
Z_EM = np.vstack(EM_model.muhats).T
Z_EM = np.vstack([np.ones((1,Z_EM.shape[1])), Z_EM])

# transform into eigen-coordinates of drift generator
e_fun_vals_EM = np.dot(np.conj(Vl_EM).T, Z_EM)

efun_idcs = np.argsort(np.real(lam_EM))[::-1]

for i in range(len(efun_idcs)):
    print('lam_{:d} = {:f} + {:f}i'.format(i, \
                                           lam_EM[efun_idcs[i]].real, \
                                           lam_EM[efun_idcs[i]].imag))

In [ ]:
ix = 0
efun_idx = efun_idcs[ix]

# color scaling
vmean = np.mean(np.reshape(np.real(e_fun_vals_EM[efun_idx,:]), (-1)))
vscl = np.max(np.absolute(np.reshape(np.real(e_fun_vals_EM[efun_idx,:]), (-1)) - vmean))
#vscl = np.max(np.reshape(np.absolute(np.real(e_fun_vals_EM[efun_idx,:])), (-1)))

plt.figure()

plt.scatter(X[0,:], \
            X[1,:], \
            c = np.real(e_fun_vals_EM[efun_idx,:]), \
            cmap = cm.seismic, \
            vmin = vmean-vscl, \
            vmax = vmean+vscl, \
            s=10)

plt.title('phi_{:d}, lam_{:d} = {:.5f} + {:.5f}i'.format(ix, ix, lam_EM[efun_idx].real, lam_EM[efun_idx].imag))

# plt.savefig('figures/Duffing/EM_dim{:d}_efun{:d}_real.png'.format(z_dim, ix), dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('figures/Duffing/EM_dim{:d}_efun{:d}_real.eps'.format(z_dim, ix), dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()


# color scaling
vscl = np.max(np.reshape(np.absolute(e_fun_vals_EM[efun_idx,:]), (-1)))

plt.figure()

plt.scatter(X[0,:], \
            X[1,:], \
            c = np.absolute(e_fun_vals_EM[efun_idx,:]), \
            cmap = cm.inferno, \
            vmin = 0, \
            vmax = vscl, \
            s=10)

plt.title('|phi_{:d}|, lam_{:d} = {:.5f} + {:.5f}i'.format(ix, ix, lam_EM[efun_idx].real, lam_EM[efun_idx].imag))

# plt.savefig('figures/Duffing/EM_dim{:d}_efun{:d}_mag.png'.format(z_dim, ix), dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('figures/Duffing/EM_dim{:d}_efun{:d}_mag.eps'.format(z_dim, ix), dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()

# plot angle of complex eigenfunction
plt.figure()

plt.scatter(X[0,:], \
            X[1,:], \
            c = np.angle(e_fun_vals_EM[efun_idx,:]), \
            cmap = cm.twilight, \
            vmin = -np.pi, \
            vmax = np.pi, \
            s=10)

plt.title('arg(phi_{:d}), lam_{:d} = {:.5f} + {:.5f}i'.format(ix, ix, lam_EM[efun_idx].real, lam_EM[efun_idx].imag))

# plt.savefig('figures/Duffing/EM_dim{:d}_efun{:d}_arg.png'.format(z_dim, ix), dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('figures/Duffing/EM_dim{:d}_efun{:d}_arg.eps'.format(z_dim, ix), dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()

# Prediction accuracy

## Load data

In [ ]:
fname = 'models/Duffing/eval_data_fullobs_fine.p'

with open(fname,'rb') as f:
    data = pickle.load(f)

X_eval = data['X_data']
Y_eval = data['Y_data']
Noise_eval = data['Noise_data']
U_eval = data['U_data']
t_eval = data['t_data']
n_traj_eval = data['n_traj']
n_t_eval = data['n_t']
dt_eval = data['dt']

In [ ]:
plt.figure()
for k in range(n_traj_eval):
    plt.plot(X_eval[k,:,0], X_eval[k,:,1], 'b.-')
    
plt.show()

## Split trajectories into observation and prediction segments

In [ ]:
print(n_t_eval)

In [ ]:
n_t_eval_obs = 200
n_t_eval_pred = n_t_eval - n_t_eval_obs

t_eval_obs = t_eval[:n_t_eval_obs]
t_eval_pred = t_eval[n_t_eval_obs:]

X_eval_obs = X_eval[:,:n_t_eval_obs,:]
X_eval_pred = X_eval[:,n_t_eval_obs:,:]

Y_eval_obs = Y_eval[:,:n_t_eval_obs,:]
Y_eval_pred = Y_eval[:,n_t_eval_obs:,:]

Noise_eval_obs = Noise_eval[:,:n_t_eval_obs,:]

U_eval_obs = U_eval[:,:n_t_eval_obs-1,:]
U_eval_pred = U_eval[:,n_t_eval_obs-1:,:]

## Load EDMD model

In [ ]:
fname = 'models/Duffing/EDMD_model_dim{:d}'.format(z_dim)
EDMD_model = ActuatedKoopmanEM()
EDMD_model, _, _ = load_EM_model(EDMD_model, fname)

## Infer latent states on observation data

In [ ]:
muhats, Sighats = EM_model.infer_latent_states(Y_eval_obs+Noise_eval_obs, dt_eval*U_eval_obs)

# muhats_EDMD, Sighats_EDMD = EDMD_model.infer_latent_states(Y_eval_obs+Noise_eval_obs, dt_eval*U_eval_obs)

## Predict dynamics

In [ ]:
## EM predictions
Y_pred, Sig_y_pred, Z_pred, Sig_z_pred = EM_model.predict_dynamics(muhats[:,-1,:], \
                                                                   Sighats[:,-1,:,:], \
                                                                   dt_eval*U_eval_pred)

## EDMD predictions
# Y_pred_EDMD, Sig_y_pred_EDMD, Z_pred_EDMD, Sig_z_pred_EDMD = EDMD_model.predict_dynamics(muhats_EDMD[:,-1,:], \
#                                                                    Sighats_EDMD[:,-1,:,:], \
#                                                                    dt_eval*U_eval_pred)

In [ ]:
## EDMD predictions

# initial conditions
Phi_eval = SpecialPolyDict(np.vstack(X_eval_obs[:,-1,:]).T/2.0, degree=deg)

# make predictions
Y_pred_EDMD = np.zeros(Y_eval_pred.shape)
for k in range(n_t_eval_pred):
    Phi_eval = Phi_eval + dt_eval*np.dot(V_EDMD.T, Phi_eval)
    Y_pred_EDMD[:,k,:] = np.transpose(np.dot(C_EDMD, Phi_eval))

In [ ]:
traj_idx = np.random.randint(n_traj_eval)

plt.figure()

plt.plot(t_eval_pred, Y_eval_pred[traj_idx,:,0], 'k-')

plt.plot(t_eval_pred, Y_pred[traj_idx,:,0], 'b-')
plt.plot(t_eval_pred, Y_pred[traj_idx,:,0]+2*Sig_y_pred[traj_idx,:,0,0], 'b--')
plt.plot(t_eval_pred, Y_pred[traj_idx,:,0]-2*Sig_y_pred[traj_idx,:,0,0], 'b--')

plt.plot(t_eval_pred, Y_pred_EDMD[traj_idx,:,0], 'g-')
# plt.plot(t_eval_pred, Y_pred_EDMD[traj_idx,:,0]+2*Sig_y_pred_EDMD[traj_idx,:,0,0], 'g--')
# plt.plot(t_eval_pred, Y_pred_EDMD[traj_idx,:,0]-2*Sig_y_pred_EDMD[traj_idx,:,0,0], 'g--')

plt.ylim([-2,2])
plt.show()

In [ ]:
fig, axs = plt.subplots(2)

# error in x
axs[0].plot(t_eval_pred, np.mean(np.square(Y_pred_EDMD[:,:,0] - Y_eval_pred[:,:,0]), axis=0), 'g-', label='EDMD')
axs[0].plot(t_eval_pred, np.mean(np.square(Y_pred[:,:,0] - Y_eval_pred[:,:,0]), axis=0), 'b-', label='EM model')
axs[0].legend(loc='lower right')

# error in dx/dt
axs[1].plot(t_eval_pred, np.mean(np.square(Y_pred_EDMD[:,:,1] - Y_eval_pred[:,:,1]), axis=0), 'g-', label='EDMD')
axs[1].plot(t_eval_pred, np.mean(np.square(Y_pred[:,:,1] - Y_eval_pred[:,:,1]), axis=0), 'b-', label='EM model')

fig.tight_layout()

# plt.savefig('figures/Duffing/dim{:d}_pred_mse.png'.format(z_dim), dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('figures/Duffing/dim{:d}_pred_mse.eps'.format(z_dim), dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()

In [ ]:
pred_errors_EM = np.square(np.linalg.norm(Y_pred - Y_eval_pred, axis=2))
pred_errors_EDMD = np.square(np.linalg.norm(Y_pred_EDMD - Y_eval_pred, axis=2))

In [ ]:
plt.figure()

plt.plot(t_eval_pred, np.mean(pred_errors_EM, axis=0), 'b-', label='EM model')
plt.plot(t_eval_pred, np.mean(pred_errors_EDMD, axis=0), 'g-', label='EDMD')
plt.legend(loc='lower right')

plt.show()